# **Simulazioni MD con il modulo CMS: CrystalStructure**

---

## Costruire un oggetto CrystalStructure

In `CMS`, un cristallo è rappresentato da un oggetto [CrystalStructure](...), che può essere istanziato a partire da un file .txt contenente le coordinate degli atomi, ad esempio: [fcc100a256.txt](../../../data/pos-fcc100-random/fcc100a256.txt).

In [1]:
from CMS.MolecularDynamics.CrystalStructure import CrystalStructure
cristallo = CrystalStructure.from_file('../../../data/pos-fcc100-random/fcc100a256.txt')

---

## Contare il numero dei vicini di ciascun atomo

Il raggio di cutoff, i primi vicini, i secondi vicini e le Verlet cages

---

## Le condizioni al contorno

Con il metodo `CrystalStructure.set_pbc()` è possibile imporre delle condizioni di periodicità ai bordi del cristallo.